# Example (Feature Engineering)

In [65]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [59]:
df = pd.read_csv('Airplane_Crashes_and_Fatalities_Since_1908.csv', sep=',')
df = df.dropna()
df =df.drop('Summary', axis=1)
df.head()

Date                        Location  \
56   1921                   Paris, France   
102  1926       Rossaugpt, Czechoslovakia   
121  1927              Hadley, New Jersey   
148  1928                 Purley, England   
150  1928  Near Port Townsend, BC, Canada   

                                            Operator  \
56                                   Franco-Roumaine   
102  Compagnie Internationale de Navigation Aérienne   
121                                 Reynolds Airways   
148                                 Imperial Airways   
150                         British Columbia Airways   

                             Route                   Type Registration  \
56   Varsovie - Strasbourg - Paris               Potez IX       F-ADCD   
102                 Paris - Prague           Caudron C-61       F-AFBT   
121                    Sightseeing           Fokker F-VII        NC776   
148                        Testing      Vickers 74 Vulcan       G-EBLB   
150             Victoria - Seattle  Ford 4-AT-B Tri Motor       G-CATX   

       cn/In  Aboard  Fatalities  Ground  
56       160     5.0         5.0     0.0  
102     5307     7.0         7.0     0.0  
121     4840    12.0         7.0     0.0  
148        9     4.0         4.0     0.0  
150  4-AT-26     7.0         7.0     0.0

In [60]:
X = df.drop('Fatalities', axis = 1)
X_number = X.select_dtypes(include='number')
X_category = X.select_dtypes(exclude='number')
Xe = (X_number - X_number.mean())/X_number.std()
X_category = pd.get_dummies(X_category)
X = pd.concat([X_number, X_category], axis =1)
Y=df.Fatalities
Y.head()

56     5.0
102    7.0
121    7.0
148    4.0
150    7.0
Name: Fatalities, dtype: float64

In [52]:
#engineer interaction features
poly = PolynomialFeatures(2, interaction_only = True, include_bias = False)
Xe = poly.fit_transform(X[1:10])
cols = poly.get_feature_names(X.columns)
Xe = pd.DataFrame(Xe, columns = cols)
Xe.head(3)

KeyboardInterrupt: 

In [ ]:
Xe = (Xe - Xe.mean())/Xe.std()

In [ ]:
Xe['bias'] = 1
Xe.head()

In [69]:
lr = LinearRegression(fit_intercept=False)
lr.fit(X, Y)
cv_results = cross_validate(lr, X, Y, return_train_score = True)
R2_train = cv_results['train_score'].mean()
R2_valid = cv_results['test_score'].mean()
y_pred = lr.predict(X)
print('Train R2', R2_train.round(2))
print('Validation R2', R2_valid.round(2))

Train R2 1.0
Validation R2 0.49


In [70]:
coef = pd.Series(lr.coef_, index = X.columns)
coef.sort_values(ascending = False).head(10)

Registration_JA8119                                    39.755897
Type_Boeing B-747-SR46                                 39.755897
cn/In_20783/230                                        39.755897
Location_Mt. Osutaka, near Ueno Village, Japan         39.755897
Location_Tenerife, Canary Islands                      32.345004
cn/In_19643/11 / 20400/157                             31.293188
Operator_Pan American World Airways / KLM              31.293188
Type_Boeing B-747-121 / Boeing B-747-206B              31.293188
Registration_N736PA/PH-BUF                             31.293188
Route_Tenerife - Las Palmas / Tenerife - Las Palmas    31.293188
dtype: float64